# NFL Data Analysis

This notebook explores the NFL data that has been loaded into our database and demonstrates some of the analyses that will be available in the app.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime

# Set plot styling
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Database connection parameters
DB_PARAMS = {
    'dbname': 'nfl_analytics',
    'user': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': '5432'
}

def get_db_connection():
    """Create a database connection with RealDictCursor for returning dictionaries."""
    conn = psycopg2.connect(**DB_PARAMS, cursor_factory=RealDictCursor)
    return conn

def execute_query(query, params=None):
    """Execute a query and return results as a DataFrame."""
    conn = get_db_connection()
    try:
        df = pd.read_sql_query(query, conn, params=params)
        return df
    finally:
        conn.close()

# Test the connection
try:
    conn = get_db_connection()
    print("Successfully connected to the database")
    conn.close()
except Exception as e:
    print(f"Error connecting to the database: {e}")

## 1. Player Performance Analysis

Let's analyze the performance of top players at different positions.

In [ ]:
def get_top_players_by_position(position, season, stat, limit=10):
    """Get top players at a position for a specific stat and season."""
    query = """
    SELECT p.id, p.name, p.position, 
           SUM(s.{stat}) as total_{stat}
    FROM players p
    JOIN player_game_stats s ON p.id = s.player_id
    JOIN games g ON s.game_key = g.key
    WHERE p.position = %s
      AND g.season = %s
    GROUP BY p.id, p.name, p.position
    ORDER BY total_{stat} DESC
    LIMIT %s
    """.format(stat=stat)
    
    return execute_query(query, params=(position, season, limit))

# Example: Top 10 QBs by passing yards in 2023
top_qbs_passing = get_top_players_by_position('QB', 2023, 'pass_yards')
top_qbs_passing

In [ ]:
# Plot top QBs passing yards
plt.figure(figsize=(12, 6))
sns.barplot(x='name', y='total_pass_yards', data=top_qbs_passing)
plt.title('Top 10 QBs by Passing Yards (2023)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 2. Position Comparison Analysis (WR1 vs WR2, RB1 vs RB2)

This is a key feature of our app - comparing performance metrics between different positional roles.

In [ ]:
def compare_positions(role1, role2, season):
    """Compare performance metrics between two roles."""
    query = """
    WITH play_stats AS (
        SELECT
            pp.role,
            pp.targeted,
            pp.completed,
            pp.yards_gained,
            p.game_key,
            p.off_team
        FROM plays p
        JOIN play_players pp ON p.id = pp.play_id
        JOIN games g ON p.game_key = g.key
        WHERE pp.role IN (%s, %s)
          AND g.season = %s
    )
    SELECT
        role,
        COUNT(*) AS total_plays,
        SUM(CASE WHEN targeted THEN 1 ELSE 0 END) AS targets,
        SUM(CASE WHEN completed THEN 1 ELSE 0 END) AS receptions,
        CASE 
            WHEN SUM(CASE WHEN targeted THEN 1 ELSE 0 END) > 0 
            THEN ROUND(SUM(CASE WHEN completed THEN 1 ELSE 0 END)::numeric / 
                 SUM(CASE WHEN targeted THEN 1 ELSE 0 END)::numeric * 100, 2)
            ELSE 0
        END AS catch_percentage,
        SUM(yards_gained) AS total_yards,
        ROUND(AVG(yards_gained) FILTER (WHERE yards_gained IS NOT NULL), 2) AS avg_yards_per_reception
    FROM play_stats
    GROUP BY role
    ORDER BY role
    """
    
    return execute_query(query, params=(role1, role2, season))

# Example: Compare WR1 vs WR2 performance in 2023
wr_comparison = compare_positions('wr1', 'wr2', 2023)
wr_comparison

In [ ]:
# Plot the comparison
metrics = ['targets', 'receptions', 'catch_percentage', 'total_yards']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    sns.barplot(x='role', y=metric, data=wr_comparison, ax=axes[i])
    axes[i].set_title(f'{metric.replace("_", " ").title()}: WR1 vs WR2')

plt.tight_layout()
plt.show()

## 3. Player Trend Analysis

Analyze how a player's performance trends over a season.

In [ ]:
def get_player_weekly_trends(player_id, season, stat):
    """Get weekly trend data for a player and stat."""
    query = """
    SELECT 
        p.name, 
        p.position,
        g.week,
        s.{stat},
        CASE 
            WHEN g.home_team = s.team THEN g.away_team 
            ELSE g.home_team 
        END AS opponent
    FROM player_game_stats s
    JOIN players p ON s.player_id = p.id
    JOIN games g ON s.game_key = g.key
    WHERE p.id = %s
      AND g.season = %s
    ORDER BY g.week
    """.format(stat=stat)
    
    return execute_query(query, params=(player_id, season))

# Example: Get weekly fantasy points for a specific player in 2023
# Replace with a valid player_id from your database
player_trend = get_player_weekly_trends('player_id_here', 2023, 'fantasy_points')
player_trend

In [ ]:
# Plot weekly trend
if not player_trend.empty:
    plt.figure(figsize=(14, 7))
    sns.lineplot(x='week', y='fantasy_points', data=player_trend, marker='o', markersize=10)
    
    player_name = player_trend['name'].iloc[0]
    position = player_trend['position'].iloc[0]
    
    plt.title(f'{player_name} ({position}) - Fantasy Points by Week (2023)')
    plt.xlabel('Week')
    plt.ylabel('Fantasy Points')
    plt.xticks(player_trend['week'])
    plt.grid(True)
    
    # Add opponent labels
    for i, row in player_trend.iterrows():
        plt.text(row['week'], row['fantasy_points'], row['opponent'], 
                 ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 4. Team Offensive Analysis

Analyze team offensive patterns and performance.

In [ ]:
def analyze_team_offense(team_code, season):
    """Analyze team offensive patterns."""
    query = """
    WITH team_plays AS (
        SELECT 
            p.pass_play,
            p.run_play,
            p.down,
            p.yards_gained,
            p.touchdown,
            p.red_zone
        FROM plays p
        JOIN games g ON p.game_key = g.key
        WHERE p.off_team = %s
          AND g.season = %s
    )
    SELECT
        COUNT(*) AS total_plays,
        SUM(CASE WHEN pass_play THEN 1 ELSE 0 END) AS pass_plays,
        SUM(CASE WHEN run_play THEN 1 ELSE 0 END) AS run_plays,
        ROUND(SUM(CASE WHEN pass_play THEN 1 ELSE 0 END)::numeric / COUNT(*)::numeric * 100, 2) AS pass_percentage,
        ROUND(SUM(CASE WHEN run_play THEN 1 ELSE 0 END)::numeric / COUNT(*)::numeric * 100, 2) AS run_percentage,
        ROUND(AVG(yards_gained), 2) AS avg_yards_per_play,
        SUM(CASE WHEN touchdown THEN 1 ELSE 0 END) AS total_touchdowns,
        SUM(CASE WHEN red_zone THEN 1 ELSE 0 END) AS red_zone_plays,
        SUM(CASE WHEN red_zone AND touchdown THEN 1 ELSE 0 END) AS red_zone_touchdowns
    FROM team_plays
    """
    
    return execute_query(query, params=(team_code, season))

# Example: Analyze a specific team's offense in 2023
# Replace with a valid team code from your database
team_offense = analyze_team_offense('team_code_here', 2023)
team_offense

In [ ]:
# Plot pass vs run distribution
if not team_offense.empty:
    data = [team_offense['pass_plays'].iloc[0], team_offense['run_plays'].iloc[0]]
    labels = ['Pass Plays', 'Run Plays']
    
    plt.figure(figsize=(10, 7))
    plt.pie(data, labels=labels, autopct='%1.1f%%', startangle=90, explode=[0.05, 0])
    plt.title(f'Team Offensive Play Distribution (2023)')
    plt.axis('equal')
    plt.show()

## 5. Advanced Situational Analysis

Analyze performance in different down and distance situations.

In [ ]:
def analyze_down_performance(team_code, season):
    """Analyze team performance by down."""
    query = """
    WITH down_plays AS (
        SELECT 
            p.down,
            p.pass_play,
            p.run_play,
            p.yards_gained,
            p.is_first_down
        FROM plays p
        JOIN games g ON p.game_key = g.key
        WHERE p.off_team = %s
          AND g.season = %s
          AND p.down IS NOT NULL
    )
    SELECT
        down,
        COUNT(*) AS total_plays,
        SUM(CASE WHEN pass_play THEN 1 ELSE 0 END) AS pass_plays,
        SUM(CASE WHEN run_play THEN 1 ELSE 0 END) AS run_plays,
        ROUND(SUM(CASE WHEN pass_play THEN 1 ELSE 0 END)::numeric / COUNT(*)::numeric * 100, 2) AS pass_percentage,
        ROUND(AVG(yards_gained), 2) AS avg_yards,
        SUM(CASE WHEN is_first_down THEN 1 ELSE 0 END) AS first_downs,
        ROUND(SUM(CASE WHEN is_first_down THEN 1 ELSE 0 END)::numeric / COUNT(*)::numeric * 100, 2) AS conversion_rate
    FROM down_plays
    GROUP BY down
    ORDER BY down
    """
    
    return execute_query(query, params=(team_code, season))

# Example: Analyze down performance for a specific team in 2023
# Replace with a valid team code from your database
down_performance = analyze_down_performance('team_code_here', 2023)
down_performance

In [ ]:
# Plot down-by-down performance
if not down_performance.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Pass vs Run percentage by down
    pass_percentage = down_performance['pass_percentage']
    run_percentage = 100 - down_performance['pass_percentage']
    
    x = np.arange(len(down_performance))
    width = 0.35
    
    ax1.bar(x, pass_percentage, width, label='Pass %')
    ax1.bar(x, run_percentage, width, bottom=pass_percentage, label='Run %')
    
    ax1.set_ylabel('Percentage')
    ax1.set_title('Pass vs Run Percentage by Down')
    ax1.set_xticks(x)
    ax1.set_xticklabels([f'{d}' for d in down_performance['down']])
    ax1.legend()
    
    # Plot 2: Conversion rate by down
    sns.barplot(x='down', y='conversion_rate', data=down_performance, ax=ax2)
    ax2.set_ylabel('First Down Conversion Rate (%)')
    ax2.set_title('Conversion Rate by Down')
    
    plt.tight_layout()
    plt.show()

## 6. Conclusion

This notebook demonstrates some of the core analysis capabilities that will be available in the NFL Analytics app. The analyses shown here include:

1. Top player performance rankings by position
2. Position role comparisons (WR1 vs WR2, etc.)
3. Player performance trends throughout the season
4. Team offensive patterns analysis
5. Situational analysis by down and distance

These analyses can be extended to build predictive models, identify player matchup advantages, and provide fantasy football insights.